In [ ]:
import os

import gym
import numpy as np
import pandas as pd
import seaborn as sns

from ray import init, rllib, tune, shutdown

In [ ]:
from attack_simulator.agents import DEFENDERS
from attack_simulator.alphazero_env import AttackSimulationAlphaZeroEnv
from attack_simulator.env import AttackSimulationEnv
from attack_simulator.graph import AttackGraph, SIZES

In [ ]:
class AgentPolicy(rllib.policy.Policy):
    def __init__(self, observation_space, action_space, config):
        super().__init__(observation_space, action_space, config)
        agent_config = dict(
            input_dim=observation_space.shape[0],
            num_actions=action_space.n,
            random_seed=config["seed"],
            attack_graph=config["env_config"]["attack_graph"],
        )
        self._agent = DEFENDERS[config["agent_type"]](agent_config)

    def compute_actions(self, observations, *args, **kwargs):
        # FIXME: use a `numpy` array as a temporary workaround for
        #        https://github.com/ray-project/ray/issues/10100
        return np.array([self._agent.act(obs) for obs in observations]), [], {}

    def get_weights(self):
        return {}

    def set_weights(self, weights):
        pass

    def learn_on_batch(self, samples):
        return {}


def template_agent(agent_type):
    default_config = rllib.agents.trainer.with_common_config(dict(agent_type=agent_type))
    return rllib.agents.trainer_template.build_trainer(
        name=agent_type,
        default_policy=AgentPolicy,
        default_config=default_config,
    )

In [ ]:
# 'contrib/AlphaZero' does NOT appear to work without its custom dense model
from ray.rllib.contrib.alpha_zero.models.custom_torch_models import DenseModel

rllib.models.ModelCatalog.register_custom_model("alpha_zero_dense_model", DenseModel)

In [ ]:
from ray.util.client import worker

worker.INITIAL_TIMEOUT_SEC = worker.MAX_TIMEOUT_SEC = 1


def ray_init():
    if os.path.isdir("/var/run/secrets/kubernetes.io") or os.path.exists(
        os.path.expanduser("~/ray_bootstrap_config.yaml")
    ):
        # inside k8s pod or a cluster managed by Ray's autoscaler
        context = init(address="auto")
    else:
        ray_client_server = "host.docker.internal" if os.path.exists("/.dockerenv") else "127.0.0.1"
        try:
            context = init(address=f"ray://{ray_client_server}:10001")
        except ConnectionError:
            # clean up after failed connection attempt
            shutdown()
            # listen on all interfaces inside a container for port-forwarding to work
            dashboard_host = "0.0.0.0" if os.path.exists("/.dockerenv") else "127.0.0.1"
            context = init(num_cpus=4, dashboard_host=dashboard_host)
    print("\x1b[33;1m", context, "\x1b[m")
    return context

In [ ]:
agent_types = ["contrib/AlphaZero", "R2D2", "rule-based", "random"]
graphs = [AttackGraph(dict(graph_size=size)) for size in SIZES]
seeds = [0, 1, 2, 3, 6, 7, 11, 28, 42, 1337]
iterations = 10
rollouts = 10

common_config = dict(
    # log_level='DEBUG',
    framework="torch",
    env=AttackSimulationEnv,
    env_config=dict(attack_graph=tune.grid_search(graphs)),
    seed=tune.grid_search(seeds),
    # common evaluation settings
    evaluation_num_workers=1,
    evaluation_config=dict(
        explore=False,
        # workaround for a bug in RLLib (https://github.com/ray-project/ray/issues/17921)
        replay_sequence_length=-1,
    ),
    evaluation_num_episodes=rollouts,
)
train_and_eval_config = dict(
    common_config,
    num_workers=2,
    # evaluation at the end
    evaluation_interval=iterations,
)
eval_only_config = dict(
    common_config,
    # evaluation ONLY: avoid MultiGPU optimizer, set all relevant sizes to 0
    simple_optimizer=True,
    num_workers=0,
    train_batch_size=0,
    rollout_fragment_length=0,
    timesteps_per_iteration=0,
    # evaluation at the end
    evaluation_interval=1,
)

In [ ]:
rename = {
    "config.env_config.agent_type": "Agent",
    "config.env_config.attack_graph": "graph",
    "evaluation.hist_stats.episode_reward": "returns",
    "evaluation.hist_stats.episode_lengths": "lengths",
}


def postprocess(results_df):
    df = results_df[rename.keys()].rename(columns=rename)
    df.dropna(inplace=True)  # remove `NaN` evaluation results from failed trials
    df["Graph size"] = df["graph"].apply(lambda g: g.num_attacks)
    del df["graph"]
    df["tuple"] = df.apply(lambda t: list(zip(t.returns, t.lengths)), axis="columns")
    del df["returns"]
    del df["lengths"]
    df = df.explode("tuple", ignore_index=True)
    df[["Returns", "Episode lengths"]] = df["tuple"].tolist()
    del df["tuple"]
    return df

In [ ]:
def generate(savename):
    ray_init()

    frames = []
    for agent_type in agent_types:
        if agent_type in DEFENDERS:
            agent = template_agent(agent_type)
            config = dict(eval_only_config)
            stop = dict(training_iteration=0)
        else:
            agent = agent_type
            if agent_type == "contrib/AlphaZero":
                config = dict(
                    train_and_eval_config,
                    env=AttackSimulationAlphaZeroEnv,
                    model=dict(custom_model="alpha_zero_dense_model"),
                    rollout_fragment_length=32,
                    train_batch_size=640,
                    buffer_size=512,
                )
            else:
                config = dict(train_and_eval_config, model=dict(use_lstm=True))

            stop = dict(training_iteration=iterations)  # TODO: additional stopping criteria?

        config["env_config"].update(agent_type=agent_type)
        results = tune.run(
            agent,
            config=config,
            stop=stop,
            max_failures=3,
            queue_trials=True,
            raise_on_failed_trial=False,
            progress_reporter=tune.JupyterNotebookReporter(overwrite=True),
            sync_config=tune.SyncConfig(sync_to_driver=False),
        )
        frames.append(results.results_df)

    shutdown()
    results_df = pd.concat(frames, ignore_index=True)
    df = postprocess(results_df)
    df.to_csv(savename)
    return df

In [ ]:
savename = "returns-tune-eval.csv"

df = generate(savename) if not os.path.exists(savename) else pd.read_csv(savename, index_col=0)

In [ ]:
sns.set(style="darkgrid", rc={"figure.figsize": (12, 8)})

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Returns", hue="Agent", ci="sd")
g.legend(title="Agent", loc="upper left")
g.set_title("Returns vs Size (random attacker)")

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Episode lengths", hue="Agent", ci="sd")
g.legend(title="Agent", loc="upper left")
g.set_title("Episode lengths vs Size (random attacker)")

In [ ]:
# pd.set_option("display.max_columns", 32)
df.groupby("Agent").describe()